In [ ]:
import numpy as np

from control_study.misc import make_sim_setup
from control_study.multibody_extras import calc_velocity_jacobian

In [ ]:
inv = np.linalg.inv
pinv = np.linalg.pinv
rank = np.linalg.matrix_rank
svd = np.linalg.svd

def calc_dynamics(plant, context):
    M = plant.CalcMassMatrix(context)
    C = plant.CalcBiasTerm(context)
    tau_g = plant.CalcGravityGeneralizedForces(context)
    return M, C, tau_g

def reproject_mass(Minv, Jt):
    _, num_dof = Jt.shape
    I_dyn = np.eye(num_dof)
    # Maps from task forces to task accelerations.
    Mtinv = Jt @ Minv @ Jt.T
    # Maps from task accelerations to task forces.
    Mt = np.linalg.inv(Mtinv)
    # Maps from task accelerations to generalized accelerations.
    # Transpose maps from generalized forces to task forces.
    Jtbar = Minv @ Jt.T @ Mt
    # Generalized force nullspace.
    Nt_T = I_dyn - Jt.T @ Jtbar.T
    return Mt, Mtinv, Jt, Jtbar, Nt_T

def spectral_norm(A):
    _, s, _ = svd(A)
    return s[0]

def np_print_more_like_matlab():
    np.set_printoptions(
        formatter={"float_kind": lambda x: f"{x: 06.3f}"},
        linewidth=150,
    )

In [ ]:
np_print_more_like_matlab()

In [ ]:
_, plant, _, frame_G = make_sim_setup(0.0)
frame_W = plant.world_frame()

In [ ]:
context = plant.CreateDefaultContext()
# q0 = np.deg2rad([0.0, 15.0, 0.0, -75.0, 0.0, 90.0, 0.0])
q0 = np.deg2rad([0.0, 0.0, 0.0, 0.0, 0.0, 90.0, 0.0])
plant.SetPositions(context, q0)

In [ ]:
M, C, tau_g = calc_dynamics(plant, context)
Minv = inv(M)
num_t = 6
num_v = 7
It = np.eye(num_t)
Iv = np.eye(num_v)
Jt = calc_velocity_jacobian(plant, context, frame_W, frame_G)
Mt, Mtinv, Jt, Jtbar, Nt_T = reproject_mass(Minv, Jt)

# Kinematic.
Jtpinv = pinv(Jt)
Nk = Iv - Jtpinv @ Jt
print(Nk)
print(Nk @ np.ones(num_v))

In [8]:
def info(name, A):
    r = rank(A)
    _, s, _ = svd(A)
    smin = np.min(s[s > 1e-10])
    print(f"{name}:")
    print(f"  rank: {r}")
    print(f"  spec: {s[0]}")
    # print(f"  spec^2: {s[0]**2}")
    # print(f"  s min: {smin}")
    # print(f"  s min inv: {1 / smin}")

info("Jt", Jt)
info("Jt.T", Jt.T)
info("Jtpinv", Jtpinv)
info("Jtbar", Jtbar)
info("Nk", Nk)
info("M", M)
info("Minv", Minv)
info("Nt_T", Nt_T)
info("Nt_T acc", Minv @ Nt_T @ M)

Jt:
  rank: 5
  spec: 1.9152197435721545
Jt.T:
  rank: 5
  spec: 1.9152197435721532
Jtpinv:
  rank: 5
  spec: 14.92053507540991
Jtbar:
  rank: 6
  spec: 15.227454881441853
Nk:
  rank: 3
  spec: 0.9999999999999999
M:
  rank: 7
  spec: 3.371259635097325
Minv:
  rank: 7
  spec: 1243.7922408554393
Nt_T:
  rank: 5
  spec: 2.048069558335394
Nt_T acc:
  rank: 5
  spec: 7.9848597236374514
